In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m-pesa-data/transanction.csv


In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Error: {e}")


✅ Gemini API key setup complete.


In [3]:
# ============================================================
# 0. CLEAN UP OLD DATABASE
# ============================================================

import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Cleaned up old database files")

# ============================================================
# 1. IMPORTS & SETUP
# ============================================================

import pandas as pd
from google.adk.agents import Agent, SequentialAgent
from google.adk.tools import FunctionTool
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# ============================================================
# 2. DATA PROCESSING TOOLS
# ============================================================

def load_data(csv_path: str = "/kaggle/input/m-pesa-data/transanction.csv") -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    numeric_cols = ["Paid In", "Withdrawn", "Balance"]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    df.dropna(subset=["Balance"], inplace=True)
    return df

def clean_and_prepare_data(df: pd.DataFrame) -> pd.DataFrame:
    df["Completion Time"] = pd.to_datetime(df['Completion Time'], errors='coerce')
    numeric_cols = ["Paid In", "Withdrawn", "Balance"]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")
    df.dropna(subset=["Balance"], inplace=True)
    return df

def sort_and_prepare_time_series(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(by="Completion Time")
    return df.set_index("Completion Time")

def categorize_transaction(details: str) -> str:
    if not isinstance(details, str):
        return "Uncategorized"
    dl = details.lower()
    if "pay bill" in dl:
        return "Bills"
    elif "mpesa" in dl or "customer payment" in dl:
        return "Income"
    elif "withdraw" in dl or "purchase" in dl:
        return "Expenses"
    else:
        return "Other"

def analyze_mpesa_data() -> dict:
    df = load_data()
    df = clean_and_prepare_data(df)
    df = sort_and_prepare_time_series(df)
    df["Category"] = df["Details"].apply(categorize_transaction)

    balance_stats = {
        "start_balance": float(df["Balance"].iloc[0]),
        "end_balance": float(df["Balance"].iloc[-1]),
        "min_balance": float(df["Balance"].min()),
        "max_balance": float(df["Balance"].max()),
        "avg_balance": float(df["Balance"].mean()),
    }

    category_spending = (
        df[df["Withdrawn"] > 0]
        .groupby("Category")["Withdrawn"]
        .sum()
        .sort_values(ascending=False)
        .to_dict()
    )

    income_total = float(df["Paid In"].sum())
    withdrawal_total = float(df["Withdrawn"].sum())

    return {
        "balance_stats": balance_stats,
        "category_spending": category_spending,
        "income_total": income_total,
        "withdrawal_total": withdrawal_total,
    }

analyze_data_tool = FunctionTool(analyze_mpesa_data)

# ============================================================
# 3. AGENTS
# ============================================================

# Agent 1: Data Analyzer (no state reference)
data_analyzer_agent = Agent(
    name="DataAnalyzerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    tools=[analyze_data_tool],
    instruction="""
You are the Data Analyzer Agent.

Your task:
- Use your tool 'analyze_data_tool' to analyze the M-Pesa statement.
- Assign the result to the session variable 'analysis_output'.

Example tool call (ADK understands this):
analysis_output = tools.call('analyze_data_tool')

Return only:
Analysis complete.
""",
    output_key="analysis_output"
)


# Agent 2: Insights Agent (reads {analysis_output})
insights_agent = Agent(
    name="InsightsAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are the Insights Agent.

You receive this data:
{analysis_output}

Tasks:
- Identify SME financial patterns
- Explain spending behavior
- Highlight risks & opportunities

Save the result into insights_output.

Return only:
Insights complete.
""",
    output_key="insights_output"
)

# Agent 3: Loan Advisor (reads {analysis_output} and {insights_output})
loan_advisor_agent = Agent(
    name="LoanAdvisorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are the Loan Advisory Agent.

You receive:
Analysis: {analysis_output}
Insights: {insights_output}

Tasks:
- Assess SME repayment ability
- Recommend suitable loan type & amount
- Provide repayment insights

Save the result into loan_advice_output.

Return only:
Loan advice complete.
""",
    output_key="loan_advice_output"
)

# ============================================================
# 4. SYSTEM PIPELINE
# ============================================================

sme_financial_system = SequentialAgent(
    name="SMEFinancialSystem",
    sub_agents=[
        data_analyzer_agent,
        insights_agent,
        loan_advisor_agent
    ]
)

runner = InMemoryRunner(agent=sme_financial_system)

# ============================================================
# 5. RUN PIPELINE
# ============================================================

response = await runner.run_debug("Analyze my statement and give me insights + loan advice")

# Print final response
print(response)


✅ Cleaned up old database files

 ### Created new session: debug_session_id

User > Analyze my statement and give me insights + loan advice


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


DataAnalyzerAgent > This is a summary of your M-Pesa statement:

**Balance Statistics:**
*   Your average balance was 155.92.
*   Your starting balance was 61.79, and your ending balance is 0.38.
*   The highest balance you reached was 960.38, and the lowest was 0.35.

**Financial Overview:**
*   Your total income was 52,769.59.
*   Your total withdrawals amounted to 55,391.65.

**Spending Insights:**
*   There is no data on spending by category.

**Loan Advice:**
Since your withdrawals exceed your income, it suggests you may be living beyond your means or relying on credit. To improve your financial health, I recommend:
*   **Budgeting:** Track your expenses to identify areas where you can cut back.
*   **Increasing Income:** Explore options for additional income streams.
*   **Reducing Debt:** Prioritize paying off any existing loans or credit.
*   **Saving:** Aim to build an emergency fund to avoid relying on loans for unexpected expenses.

If you're considering a loan, carefully as